# Donkey Colab 3.1.0 with GPU(TensorFlow 1.13.1)

Donkey Colab with GPU.
- Version 0.1(2019年9月3日)


## 概要
Google Colabを使用し、Donkey Carの学習を行います

上からコードをを実行することで、以下のことができます
- Google Colabの環境設定
- 学習
- 学習結果の取得  




# TensorFlowのインストール

TensorFlowのVersionは、DonkeyCarのRaspberryPi3にはいっているバージョンと同じバージョンをインストールします。  
Google Colabはデフォルトでtensorflow-1.14.0が入っています（2019年9月3日）。  
なので一度アンインストールし、バージョンを揃えています。

In [0]:
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.13.1

Uninstalling tensorflow-1.14.0:
  Successfully uninstalled tensorflow-1.14.0
     |████████████████████████████████| 345.2MB 56kB/s 
     |████████████████████████████████| 368kB 42.2MB/s 
     |████████████████████████████████| 3.2MB 44.8MB/s 
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0


# TensorFlowのバージョン確認

In [0]:
!pip freeze | grep -i -e tensorflow

mesh-tensorflow==0.0.5
tensorflow-estimator==1.13.0
tensorflow-gpu==1.13.1
tensorflow-hub==0.5.0
tensorflow-metadata==0.14.0
tensorflow-probability==0.7.0


# GPUの確認

"Found GPU at: /device:GPU:0"と表示されれば、GPUが使用できる状態です。

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# DonkeyCarのインストール

In [0]:
!git clone https://github.com/autorope/donkeycar.git 
%cd donkeycar
!git checkout master
!pip3 install -e .[pc]

Cloning into 'donkeycar'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 10634 (delta 21), reused 15 (delta 3), pack-reused 10581
Receiving objects: 100% (10634/10634), 58.77 MiB | 27.06 MiB/s, done.
Resolving deltas: 100% (6561/6561), done.
/content/donkeycar
Branch 'master' set up to track remote branch 'master' from 'origin'.
Switched to a new branch 'master'
Obtaining file:///content/donkeycar
     |████████████████████████████████| 92kB 7.2MB/s 
  Created wheel for paho-mqtt: filename=paho_mqtt-1.4.0-cp36-none-any.whl size=48331 sha256=f09ae335dc310e8dd1cc00aa780aeb86115d5a25cc3d158734ab818b17473384
  Stored in directory: /root/.cache/pip/wheels/82/e5/de/d90d0f397648a1b58ffeea1b5742ac8c77f71fd43b550fa5a5
Successfully built paho-mqtt
  Running setup.py develop for donkeycar


# DonkeyCarのProjectを作成

In [0]:
!donkey createcar --path /content/mycar

using donkey v3.1.0 ...
Creating car folder: /content/mycar
making dir  /content/mycar
Creating data & model folders.
making dir  /content/mycar/models
making dir  /content/mycar/data
making dir  /content/mycar/logs
Copying car application template: complete
Copying car config defaults. Adjust these before starting your car.
Copying train script. Adjust these before starting your car.
Copying my car config overrides
Donkey setup complete.


# datas.zipのアップロードとデータの解凍
datas.zip（DonkeyCarで集めた学習用データ）をアップロードします。  
下記コードを実行すると、ファイルのアップロード フォームが出現します。  

ファイルは`/content/mycar/data/`以下に送信され、datas.zipの解凍を行っています

In [0]:
import os
from google.colab import files

if(os.path.exists("/content/datas.zip")):
   os.remove("/content/datas.zip")
if(os.path.exists("/content/mycar/data/datas.zip")):
   os.remove("/content/mycar/data/datas.zip")

%cd /content/mycar/data/
uploaded = files.upload()

!unzip -o datas.zip

/content/mycar/data


Saving datas.zip to datas.zip
Archive:  datas.zip
   creating: tub_3_19-09-03/
  inflating: tub_3_19-09-03/1489_cam-image_array_.jpg  
  inflating: tub_3_19-09-03/2835_cam-image_array_.jpg  
  inflating: tub_3_19-09-03/record_1917.json  
  inflating: tub_3_19-09-03/488_cam-image_array_.jpg  
  inflating: tub_3_19-09-03/2634_cam-image_array_.jpg  
  inflating: tub_3_19-09-03/2093_cam-image_array_.jpg  
  inflating: tub_3_19-09-03/1280_cam-image_array_.jpg  
  inflating: tub_3_19-09-03/1051_cam-image_array_.jpg  
  inflating: tub_3_19-09-03/2321_cam-image_array_.jpg  
  inflating: tub_3_19-09-03/record_1948.json  
  inflating: tub_3_19-09-03/record_2556.json  
  inflating: tub_3_19-09-03/record_2198.json  
  inflating: tub_3_19-09-03/1580_cam-image_array_.jpg  
  inflating: tub_3_19-09-03/record_456.json  
  inflating: tub_3_19-09-03/record_1895.json  
  inflating: tub_3_19-09-03/record_908.json  
  inflating: tub_3_19-09-03/record_1967.json  
  inflating: tub_3_19-09-03/record_1806.json

#学習の開始

Colabの環境構築が出来たので、DonkeyCarの学習を行います


In [0]:
%cd /content/mycar
!python manage.py train --model=/content/mycar/models/mypilot.h5 --type=linear

/content/mycar
using donkey v3.1.0 ...
loading config file: /content/mycar/config.py
loading personal config over-rides

config loaded
2019-09-03 07:31:31.447187: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-09-03 07:31:31.508465: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-09-03 07:31:31.509433: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2cbb1e0 executing computations on platform CUDA. Devices:
2019-09-03 07:31:31.509469: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla K80, Compute Capability 3.7
2019-09-03 07:31:31.511679: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-09-03 07:31:31.511905: I tensorflow/compiler/xla/service/ser

# 学習結果の転送
学習結果は、/content/mycar/models/mypilot.h5に保存されます。  
mypilot.h5をダブルクリックすると学習結果をダウンロードできます。  
RaspberryPiの~/mycar/models/mypilot.h5に学習結果をコピーすることで、DonkeyCarの自動走行ができるようになります。

![](https://faboplatform.github.io/DonkeyDocs/0.DonkeyCar3%E3%81%AE%E6%A7%8B%E7%AF%89/img/upload.png)